In [1]:
import sys
!"{sys.executable}" -m pip install customtkinter openai langchain langchain-openai


   ---------------------------------------- 0/2 [darkdetect]
   -------------------- ------------------- 1/2 [customtkinter]
   -------------------- ------------------- 1/2 [customtkinter]
   -------------------- ------------------- 1/2 [customtkinter]
   -------------------- ------------------- 1/2 [customtkinter]
   -------------------- ------------------- 1/2 [customtkinter]
   ---------------------------------------- 2/2 [customtkinter]




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import sys
!"{sys.executable}" -m pip install tkcalendar


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached babel-2.17.0-py3-none-any.whl.metadata (2.0 kB)
Using cached babel-2.17.0-py3-none-any.whl (10.2 MB)

   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------

In [1]:
import customtkinter as ctk
from tkinter import messagebox
from tkcalendar import DateEntry
from PIL import Image
from datetime import datetime
from langchain_openai import ChatOpenAI
import os
import re

# Initialize LangChain LLM
llm = ChatOpenAI(model="gpt-4o-mini")  # requires OPENAI_API_KEY set in environment

# =============== Setup ===============
ctk.set_appearance_mode("System")
ctk.set_default_color_theme("blue")

app = ctk.CTk()
app.title("Personalized Activity Guide")
app.geometry("650x700")

# =============== Title + Icon ===============
header_frame = ctk.CTkFrame(app)
header_frame.pack(pady=15)

# Convert your calendar.svg → calendar.png before
calendar_img = ctk.CTkImage(
    light_image=Image.open("calendar.png"),
    dark_image=Image.open("calendar.png"),
    size=(50, 50)
)

calendar_label = ctk.CTkLabel(header_frame, image=calendar_img, text="")
calendar_label.pack(side="left", padx=10)

title = ctk.CTkLabel(
    header_frame,
    text="Personalized Activity Guide",
    font=("Arial", 22, "bold")
)
title.pack(side="left")

subtitle = ctk.CTkLabel(
    app,
    text="Generates custom guest guides based on stay details, interests, and transportation preferences.",
    wraplength=500,
    justify="center"
)
subtitle.pack(pady=5)

# =============== Guest Info ===============
guest_frame = ctk.CTkFrame(app)
guest_frame.pack(pady=10, padx=20, fill="x")

fname = ctk.CTkEntry(guest_frame, placeholder_text="First Name", width=150)
fname.grid(row=0, column=0, padx=5, pady=5)

lname = ctk.CTkEntry(guest_frame, placeholder_text="Last Name", width=150)
lname.grid(row=0, column=1, padx=5, pady=5)

guests = ctk.CTkEntry(guest_frame, placeholder_text="# Guests", width=80)
guests.grid(row=0, column=2, padx=5, pady=5)

destination = ctk.CTkEntry(guest_frame, placeholder_text="Destination", width=200)
destination.grid(row=0, column=3, padx=5, pady=5)

# =============== Date Selection ===============
date_frame = ctk.CTkFrame(app)
date_frame.pack(pady=10, padx=20, fill="x")

checkin_label = ctk.CTkLabel(date_frame, text="Check In:")
checkin_label.grid(row=0, column=0, padx=5)
checkin = DateEntry(date_frame, width=12, background='darkblue', foreground='white', borderwidth=2)
checkin.grid(row=0, column=1, padx=5, pady=5)

checkout_label = ctk.CTkLabel(date_frame, text="Check Out:")
checkout_label.grid(row=0, column=2, padx=5)
checkout = DateEntry(date_frame, width=12, background='darkblue', foreground='white', borderwidth=2)
checkout.grid(row=0, column=3, padx=5, pady=5)

# =============== Preferences ===============
prefs_frame = ctk.CTkFrame(app)
prefs_frame.pack(pady=10, padx=20, fill="x")

activity_label = ctk.CTkLabel(prefs_frame, text="Activity Preferences", font=("Arial", 14, "bold"))
activity_label.grid(row=0, column=0, sticky="w", padx=20)

trans_label = ctk.CTkLabel(prefs_frame, text="Transportation Preferences", font=("Arial", 14, "bold"))
trans_label.grid(row=0, column=1, sticky="w", padx=20)

# Activity Checkboxes
activities = ["Relaxation", "Adventure", "Family-Friendly", "Cultural", "Food & Drink"]
activity_vars = {}
for i, act in enumerate(activities):
    var = ctk.BooleanVar()
    chk = ctk.CTkCheckBox(prefs_frame, text=act, variable=var)
    chk.grid(row=i+1, column=0, sticky="w", padx=20)
    activity_vars[act] = var

# Transport Checkboxes
transports = ["Car Rental", "Ride Share", "Metro / Bus", "Bike", "Walk"]
transport_vars = {}
for i, trans in enumerate(transports):
    var = ctk.BooleanVar()
    chk = ctk.CTkCheckBox(prefs_frame, text=trans, variable=var)
    chk.grid(row=i+1, column=1, sticky="w", padx=20)
    transport_vars[trans] = var

# =============== Generate Guide ===============
def generate_guide():
    try:
        # Get form inputs
        first_name = fname.get().strip()
        last_name = lname.get().strip()
        guest_count = guests.get()
        checkin_date = checkin.get_date()
        checkout_date = checkout.get_date()
        destination_value = destination.get().strip()

        # Validate date range
        if checkout_date <= checkin_date:
            messagebox.showerror("Invalid Dates", "❌ Check-out must be after check-in.")
            return

        # Compute days
        days = (checkout_date - checkin_date).days

        # Collect selected activities & transport
        selected_activities = [a for a, v in activity_vars.items() if v.get()]
        selected_transports = [t for t, v in transport_vars.items() if v.get()]

        # Build AI prompt
        prompt = f"""
        Create a travel activity guide for {destination_value} lasting {days} days.
        Include daily recommendations for sightseeing, food, and culture.
        Traveler: {first_name} {last_name}
        Number of Guests: {guest_count}
        Activities Chosen: {", ".join(selected_activities) or "None"}
        Transport Preferences: {", ".join(selected_transports) or "None"}
        """

        # Call LLM
        result = llm.invoke(prompt).content.strip()

        # Save to Markdown file
        filename = f"travel-guide-output/travel_guide_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(result)

        # Show result in a new window
        result_window = ctk.CTkToplevel(app)
        result_window.title("Generated Guide")
        result_window.geometry("600x500")

        result_box = ctk.CTkTextbox(result_window, width=550, height=400, wrap="word")
        result_box.pack(pady=15, padx=15, fill="both", expand=True)
        result_box.insert("1.0", result)
        result_box.configure(state="disabled")  # make read-only

        # Success message
        messagebox.showinfo("Saved", f"✅ Travel guide saved as {filename}")
        print(f"Guide saved as {filename}")
        app.destroy()

    except Exception as e:
        messagebox.showerror("Error", f"⚠️ Something went wrong:\n{str(e)}")
        app.destroy()

btn = ctk.CTkButton(app, text="⚡ Generate Guide", command=generate_guide)
btn.pack(pady=20)

app.mainloop()


### Monochrome design 

In [2]:
import sys
!"{sys.executable}" -m pip install markdown tkhtmlview

  Using cached pillow-10.4.0-cp310-cp310-win_amd64.whl.metadata (9.3 kB)
Using cached pillow-10.4.0-cp310-cp310-win_amd64.whl (2.6 MB)

  Attempting uninstall: Pillow

    Found existing installation: pillow 11.3.0

    Uninstalling pillow-11.3.0:

      Successfully uninstalled pillow-11.3.0

   ---------------------------------------- 0/2 [Pillow]
   ---------------------------------------- 0/2 [Pillow]
   ---------------------------------------- 0/2 [Pillow]
   ---------------------------------------- 0/2 [Pillow]
   ---------------------------------------- 0/2 [Pillow]
   ---------------------------------------- 0/2 [Pillow]
   ---------------------------------------- 0/2 [Pillow]
   ---------------------------------------- 0/2 [Pillow]
   ---------------------------------------- 0/2 [Pillow]
   ---------------------------------------- 0/2 [Pillow]
   ---------------------------------------- 0/2 [Pillow]
   ---------------------------------------- 0/2 [Pillow]
   ------------------

  You can safely remove it manually.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import customtkinter as ctk
from tkinter import messagebox
from tkcalendar import DateEntry
from datetime import datetime
from langchain_openai import ChatOpenAI
import os
import markdown
import tkinterweb as HtmlFrame

# Initialize LangChain LLM
llm = ChatOpenAI(model="gpt-4o-mini")  # requires OPENAI_API_KEY

# =============== Setup ===============
ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")

COLORS = {
    'bg_primary': '#f7f7f7',
    'bg_secondary': '#ededed',
    'bg_tertiary': '#d6d6d6',
    'text_primary': '#222222',
    'text_secondary': '#555555',
    'accent': '#b0b0b0',
    'border': '#cccccc'
}

app = ctk.CTk()
app.title("Personalized Activity Guide")
app.geometry("1200x900")
app.configure(fg_color=COLORS['bg_primary'])

# Ensure output folder exists
os.makedirs("travel-guide-output", exist_ok=True)

# =============== Main Container ===============
main_container = ctk.CTkScrollableFrame(
    app,
    fg_color=COLORS['bg_primary'],
    scrollbar_fg_color=COLORS['bg_secondary']
)
main_container.pack(fill="both", expand=True, padx=20, pady=20)

# =============== Header Section ===============
header_frame = ctk.CTkFrame(
    main_container,
    fg_color=COLORS['bg_secondary'],
    corner_radius=15,
    border_width=1,
    border_color=COLORS['border']
)
header_frame.pack(fill="x", pady=(0, 20))

title = ctk.CTkLabel(
    header_frame,
    text="✈ PERSONALIZED ACTIVITY GUIDE",
    font=("SF Pro Display", 28, "bold"),
    text_color=COLORS['text_primary']
)
title.pack(pady=(20, 5))

subtitle = ctk.CTkLabel(
    header_frame,
    text="Create custom travel itineraries tailored to your preferences",
    font=("SF Pro Display", 14),
    text_color=COLORS['text_secondary'],
    wraplength=600
)
subtitle.pack(pady=(0, 20))

# =============== Progress Bar ===============
progress_frame = ctk.CTkFrame(main_container, fg_color=COLORS["bg_secondary"])
progress_frame.pack(fill="x", padx=20, pady=10)

progress_bar = ctk.CTkProgressBar(
    progress_frame,
    progress_color=COLORS["accent"],
    fg_color=COLORS["bg_tertiary"],
    height=15
)
progress_bar.pack(fill="x", padx=10, pady=5)
progress_bar.set(0)

current_step = 0
total_steps = 7  # last name → guests → destination → check-in → check-out → activity → transportation

progress = ctk.CTkProgressBar(main_container, mode="indeterminate")
progress.pack_forget()

def start_progress(msg="Loading..."):
    """Show and start progress bar with a message."""
    progress.pack(pady=10)
    progress.start()
    app.update()

def stop_progress():
    """Stop and hide progress bar."""
    progress.stop()
    progress.pack_forget()
    app.update()

def complete_step():
    """Increment progress bar by one step"""
    global current_step
    if current_step < total_steps:
        current_step += 1
        progress_bar.set(current_step / total_steps)

def on_guests_focus_out(event):
    if guests.get().strip():
        complete_step()

def on_destination_focus_out(event):
    if destination.get().strip():
        complete_step()

# =============== Guest Section ===============
guest_section = ctk.CTkFrame(main_container, fg_color=COLORS['bg_secondary'],
                             corner_radius=15, border_width=1, border_color=COLORS['border'])
guest_section.pack(fill="x", pady=(0, 15))

guest_title = ctk.CTkLabel(guest_section, text="GUEST INFORMATION",
                           font=("SF Pro Display", 16, "bold"), text_color=COLORS['text_primary'])
guest_title.pack(pady=(15, 10))

guest_grid = ctk.CTkFrame(guest_section, fg_color="transparent")
guest_grid.pack(padx=20, pady=(0, 15))
guest_grid.grid_columnconfigure((0, 1, 2, 3), weight=1)

fname = ctk.CTkEntry(guest_grid, placeholder_text="First Name", height=40,
                     font=("SF Pro Display", 12), fg_color=COLORS['bg_tertiary'],
                     border_color=COLORS['border'], text_color=COLORS['text_primary'])
fname.grid(row=0, column=0, padx=(0, 8), pady=5, sticky="ew")

lname = ctk.CTkEntry(guest_grid, placeholder_text="Last Name", height=40,
                     font=("SF Pro Display", 12), fg_color=COLORS['bg_tertiary'],
                     border_color=COLORS['border'], text_color=COLORS['text_primary'])
lname.grid(row=0, column=1, padx=4, pady=5, sticky="ew")

guests = ctk.CTkEntry(guest_grid, placeholder_text="Guests", height=40,
                      font=("SF Pro Display", 12), fg_color=COLORS['bg_tertiary'],
                      border_color=COLORS['border'], text_color=COLORS['text_primary'])
guests.grid(row=0, column=2, padx=4, pady=5, sticky="ew")
guests.bind("<FocusOut>", on_guests_focus_out)

destination = ctk.CTkEntry(guest_grid, placeholder_text="Destination City", height=40,
                           font=("SF Pro Display", 12), fg_color=COLORS['bg_tertiary'],
                           border_color=COLORS['border'], text_color=COLORS['text_primary'])
destination.grid(row=0, column=3, padx=(8, 0), pady=5, sticky="ew")
destination.bind("<FocusOut>", on_destination_focus_out)

# =============== Dates Section ===============
date_section = ctk.CTkFrame(main_container, fg_color=COLORS['bg_secondary'],
                            corner_radius=15, border_width=1, border_color=COLORS['border'])
date_section.pack(fill="x", pady=(0, 15))

date_title = ctk.CTkLabel(date_section, text="TRAVEL DATES",
                          font=("SF Pro Display", 16, "bold"), text_color=COLORS['text_primary'])
date_title.pack(pady=(15, 10))

date_grid = ctk.CTkFrame(date_section, fg_color="transparent")
date_grid.pack(padx=40, pady=(0, 15))
date_grid.grid_columnconfigure((0, 1, 2, 3), weight=1)

checkin = DateEntry(date_grid, width=15, background='#2d2d2d', foreground='white',
                    borderwidth=1, font=("SF Pro Display", 11))
checkin.grid(row=1, column=0, padx=10, pady=5)
checkin.bind("<<DateEntrySelected>>", lambda e: complete_step())

checkout = DateEntry(date_grid, width=15, background='#2d2d2d', foreground='white',
                     borderwidth=1, font=("SF Pro Display", 11))
checkout.grid(row=1, column=2, padx=10, pady=5)
checkout.bind("<<DateEntrySelected>>", lambda e: complete_step())

# =============== Preferences Section ===============
prefs_section = ctk.CTkFrame(main_container, fg_color=COLORS['bg_secondary'],
                             corner_radius=15, border_width=1, border_color=COLORS['border'])
prefs_section.pack(fill="x", pady=(0, 15))

prefs_grid = ctk.CTkFrame(prefs_section, fg_color="transparent")
prefs_grid.pack(padx=20, pady=(0, 20), fill="x")
prefs_grid.grid_columnconfigure((0, 1), weight=1)

# Activities
activity_frame = ctk.CTkFrame(prefs_grid, fg_color=COLORS['bg_tertiary'], corner_radius=10)
activity_frame.grid(row=0, column=0, padx=(0, 10), pady=5, sticky="ew")

activity_label = ctk.CTkLabel(activity_frame, text="Activity Interests",
                              font=("SF Pro Display", 14, "bold"), text_color=COLORS['text_primary'])
activity_label.pack(pady=(15, 10))

activities = ["🏖 Relaxation", "🏔 Adventure", "👨‍👩‍👧‍👦 Family-Friendly", "🏛 Cultural", "🍽 Food & Drink"]
activity_vars = {}
for act in activities:
    var = ctk.BooleanVar()
    chk = ctk.CTkCheckBox(activity_frame, text=act, variable=var,
                          font=("SF Pro Display", 12), text_color=COLORS['text_primary'],
                          fg_color=COLORS['accent'], hover_color=COLORS['text_secondary'],
                          checkmark_color=COLORS['bg_primary'], command=complete_step)
    chk.pack(pady=3, padx=15, anchor="w")
    activity_vars[act] = var

# Transport
transport_frame = ctk.CTkFrame(prefs_grid, fg_color=COLORS['bg_tertiary'], corner_radius=10)
transport_frame.grid(row=0, column=1, padx=(10, 0), pady=5, sticky="ew")

transport_label = ctk.CTkLabel(transport_frame, text="Transportation",
                               font=("SF Pro Display", 14, "bold"), text_color=COLORS['text_primary'])
transport_label.pack(pady=(15, 10))

transports = ["🚗 Car Rental", "🚕 Ride Share", "🚇 Metro / Bus", "🚲 Bike", "🚶 Walk"]
transport_vars = {}
for trans in transports:
    var = ctk.BooleanVar()
    chk = ctk.CTkCheckBox(transport_frame, text=trans, variable=var,
                          font=("SF Pro Display", 12), text_color=COLORS['text_primary'],
                          fg_color=COLORS['accent'], hover_color=COLORS['text_secondary'],
                          checkmark_color=COLORS['bg_primary'], command=complete_step)
    chk.pack(pady=3, padx=15, anchor="w")
    transport_vars[trans] = var

# =============== Generate Guide ===============
def generate_guide():
    try:
        complete_step()
        start_progress("Fetching guide...")
        first_name = fname.get().strip()
        last_name = lname.get().strip()
        guest_count = guests.get()
        checkin_date = checkin.get_date()
        checkout_date = checkout.get_date()
        destination_value = destination.get().strip()

        if not first_name or not destination_value:
            messagebox.showerror("Missing Information", "❌ Please fill in at least First Name and Destination.")
            return
        if checkout_date <= checkin_date:
            messagebox.showerror("Invalid Dates", "❌ Check-out must be after check-in.")
            return

        days = (checkout_date - checkin_date).days
        selected_activities = [a for a, v in activity_vars.items() if v.get()]
        selected_transports = [t for t, v in transport_vars.items() if v.get()]

        prompt = f"""
        Create a comprehensive travel activity guide for {destination_value} lasting {days} days.
        Include daily recommendations for sightseeing, dining, and cultural experiences.
        Traveler: {first_name} {last_name}, Party Size: {guest_count}, Dates: {checkin_date} → {checkout_date}
        Activities: {", ".join(selected_activities) or "Open to all experiences"}
        Transportation: {", ".join(selected_transports) or "Flexible"}
        """

        btn.configure(text="🔄 Generating Guide...", state="disabled")
        app.update()

        result = llm.invoke(prompt).content.strip()

        filename = f"travel-guide-output/travel_guide_{first_name}_{last_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(result)

        show_result_window(result, filename, f"{first_name} {last_name}", destination_value, days)
        messagebox.showinfo("Saved", f"✅ Travel guide saved as {filename}")

        btn.configure(text="✨ Generate Guide", state="normal")
        stop_progress()

    except Exception as e:
        btn.configure(text="✨ Generate Guide", state="normal")
        messagebox.showerror("Error", f"⚠️ Something went wrong:\n{e}")

# =============== Result Window with WYSIWYG ===============
def show_result_window(content, filename, name, destination, days):
    win = ctk.CTkToplevel(app)
    win.title(f"Travel Guide - {name}")
    win.geometry("950x750")

    header = ctk.CTkLabel(win, text=f"📝 Editable Guide for {name} ({destination}, {days} days)",
                          font=("Helvetica", 18, "bold"))
    header.pack(pady=(10, 5))

    toolbar = ctk.CTkFrame(win, fg_color="transparent")
    toolbar.pack(fill="x", padx=10, pady=(0, 5))

    editor = HtmlFrame.HtmlFrame(win, horizontal_scrollbar="auto")
    editor.pack(fill="both", expand=True, padx=15, pady=15)

    html_body = markdown.markdown(content, extensions=["extra", "nl2br", "sane_lists"])
    html_template = f"""<html><body><div id="editor" contenteditable="true">{html_body}</div></body></html>"""
    editor.load_html(html_template)

    def run_js(cmd):
        try:
            editor.htmlframe.runjavascript(f"document.execCommand('{cmd}', false, null);")
        except Exception as e:
            print("JS Error:", e)

    for text, cmd in [("B", "bold"), ("I", "italic"), ("• List", "insertUnorderedList"), ("1. List", "insertOrderedList")]:
        ctk.CTkButton(toolbar, text=text, width=60, command=lambda c=cmd: run_js(c)).pack(side="left", padx=2)

    def save_content():
        html_data = editor.html
        with open(filename, "w", encoding="utf-8") as f:
            f.write(html_data)
        print(f"✅ Saved to {filename}")

    ctk.CTkButton(win, text="💾 Save", command=save_content).pack(pady=10)
    ctk.CTkButton(win, text="✕ Close", command=win.destroy).pack(pady=5)

# =============== Generate Button ===============
btn = ctk.CTkButton(main_container, text="✨ Generate Guide", command=generate_guide,
                    font=("SF Pro Display", 16, "bold"), height=50, width=200,
                    fg_color=COLORS['text_primary'], hover_color=COLORS['text_secondary'],
                    text_color=COLORS['bg_primary'], corner_radius=8)
btn.pack(pady=20)

footer = ctk.CTkLabel(main_container, text="Digital Mixology",
                      font=("SF Pro Display", 10), text_color=COLORS['text_secondary'])
footer.pack(pady=(10, 0))

# =============== Validation & Shortcuts ===============
def validate_numeric_input(event):
    """Only allow numbers in Guests field"""
    current = guests.get()
    if current and not current.isdigit():
        guests.delete(len(current)-1)

guests.bind('<KeyRelease>', validate_numeric_input)
app.bind('<Return>', lambda e: generate_guide())

app.resizable(True, True)
app.minsize(600, 700)
app.mainloop()


Starting TkinterWeb for Intel64 Family 6 Model 158 Stepping 11, GenuineIntel Windows with Python 3.10.6

Using Tkhtml 3.1 because it is already loaded

Tkhtml 3.1 successfully loaded from c:\Users\WINDOWS 10\AppData\Local\Programs\Python\Python310\lib\site-packages\tkinterweb_tkhtml\tkhtml

Welcome to TkinterWeb!
                                
The API changed in version 4. See https://tkinterweb.readthedocs.io/ for details.

Debugging messages are enabled. Use the parameter `messages_enabled = False` when calling HtmlFrame() or HtmlLabel() to disable these messages.
                                
Load about:tkinterweb for debugging information.
                                
If you enjoyed using this package, consider donating at https://buymeacoffee.com/andereoo!



## Travel guide using Nice GUI

Note: you cannot run the gui inside the jupyter. It must be run in the cmd: python dynamic-travel.py


In [ ]:
from nicegui import ui
from datetime import datetime
import json, os, anyio
from typing import Any, Dict
from langchain_openai import ChatOpenAI

# ===== Lazy LLM =====
def get_llm():
    return ChatOpenAI(model="gpt-4o-mini")  # requires OPENAI_API_KEY

os.makedirs("travel-guide-output", exist_ok=True)

# ===== Example Config =====
CONFIG_JSON = """
{
  "title": "✈ PERSONALIZED ACTIVITY GUIDE",
  "subtitle": "Create custom travel itineraries tailored to your preferences",
  "sections": [
    {
      "title": "GUEST INFORMATION",
      "layout": "row",
      "fields": [
        {"id": "first_name", "label": "First Name", "type": "text", "width": "w-1/4"},
        {"id": "last_name", "label": "Last Name", "type": "text", "width": "w-1/4"},
        {"id": "guests", "label": "Guests", "type": "number", "width": "w-1/6", "default": 2, "props": {"min": 1}},
        {"id": "destination", "label": "Destination City", "type": "text", "width": "w-1/3"}
      ]
    },
    {
      "title": "TRAVEL DATES",
      "layout": "row",
      "fields": [
        {"id": "check_in", "label": "Check In", "type": "date", "width": "w-1/2"},
        {"id": "check_out", "label": "Check Out", "type": "date", "width": "w-1/2"}
      ]
    },
    {
      "title": "PREFERENCES",
      "layout": "two_cards",
      "cards": [
        {
          "title": "Activity Interests",
          "fields": [
            {"id": "relaxation", "label": "Relaxation", "type": "checkbox"},
            {"id": "adventure", "label": "Adventure", "type": "checkbox"},
            {"id": "family", "label": "Family-Friendly", "type": "checkbox"},
            {"id": "cultural", "label": "Cultural", "type": "checkbox"},
            {"id": "food", "label": "Food & Drink", "type": "checkbox"}
          ]
        },
        {
          "title": "Transportation",
          "fields": [
            {"id": "car", "label": "Car Rental", "type": "checkbox"},
            {"id": "rideshare", "label": "Ride Share", "type": "checkbox"},
            {"id": "metro", "label": "Metro / Bus", "type": "checkbox"},
            {"id": "bike", "label": "Bike", "type": "checkbox"},
            {"id": "walk", "label": "Walk", "type": "checkbox"}
          ]
        }
      ]
    }
  ],
  "primary_action": {"label": "Generate Guide"}
}
"""

# ===== Dynamic Renderer =====
WidgetRef = Dict[str, Any]

def render_field(field: Dict[str, Any], widgets: WidgetRef):
    t = field.get("type", "text")
    label = field.get("label", "")
    width = field.get("width", "w-full")
    default = field.get("default")
    props = field.get("props", {})

    if t == "text":
        w = ui.input(label, value=default).classes(width)
    elif t == "number":
        min_val = props.get("min", None)
        max_val = props.get("max", None)
        w = ui.number(label, value=default, min=min_val, max=max_val).classes(width)
    elif t == "date":
        w = ui.input(label).props("type=date").classes(width)
    elif t == "checkbox":
        w = ui.checkbox(label, value=bool(default))
    else:
        w = ui.input(label, value=default).classes(width)

    widgets[field["id"]] = w
    return w

def collect_values(widgets: WidgetRef) -> Dict[str, Any]:
    return {fid: getattr(w, "value", None) for fid, w in widgets.items()}

# ===== Shared result state =====
result_data = {"content": "", "title": ""}

# ===== Pages =====
def build_ui(config: Dict[str, Any], widgets: WidgetRef):
    with ui.card().classes("max-w-3xl mx-auto p-6 space-y-2"):
        ui.label(config.get("title", "Dynamic Form")).classes("text-2xl font-bold text-center")
        if config.get("subtitle"):
            ui.label(config["subtitle"]).classes("text-sm text-center text-gray-500")

        for i, section in enumerate(config.get("sections", [])):
            if i > 0: ui.separator()
            ui.label(section.get("title", "")).classes("text-lg font-semibold text-center")

            layout = section.get("layout", "row")
            if layout == "row":
                with ui.row().classes("w-full justify-between"):
                    for f in section.get("fields", []):
                        render_field(f, widgets)
            elif layout == "two_cards":
                with ui.row().classes("grid grid-cols-1 md:grid-cols-2 gap-4 w-full"):
                    for card in section.get("cards", []):
                        with ui.card().classes("p-3"):
                            ui.label(card["title"]).classes("font-semibold")
                            for f in card.get("fields", []):
                                render_field(f, widgets)

        action_label = config.get("primary_action", {}).get("label", "Submit")
        ui.button(action_label, on_click=lambda: generate_guide(widgets)).classes(
            "w-full bg-black text-white mt-4 p-3 rounded-lg"
        )


@ui.page("/result")
def result_page():
    ui.label(result_data["title"]).classes("text-xl font-bold")
    ui.markdown(result_data["content"]).classes("prose max-w-none")
    ui.button("⬅ Back", on_click=lambda: ui.navigate.to("/"))

    if result_data.get("show_dialog"):
        with ui.dialog() as msg, ui.card():
            ui.label("📥 Your guide has been saved as Markdown").classes("text-lg")
            ui.button("OK", on_click=msg.close).classes("bg-blue-500 text-white")
        msg.open()
        result_data["show_dialog"] = False   # reset

# ===== Guide Generation =====
async def generate_guide(widgets: WidgetRef):
    values = collect_values(widgets)
    fname = values.get("first_name") or ""
    lname = values.get("last_name") or ""
    dest = values.get("destination") or ""
    guests = values.get("guests") or "N/A"
    check_in = values.get("check_in") or ""
    check_out = values.get("check_out") or ""

    if not fname or not dest:
        ui.notify("❌ Please fill in at least First Name and Destination.", color="negative")
        return

    try:
        checkin_date = datetime.fromisoformat(check_in) if check_in else None
        checkout_date = datetime.fromisoformat(check_out) if check_out else None
    except Exception:
        checkin_date = checkout_date = None

    if checkin_date and checkout_date and checkout_date <= checkin_date:
        ui.notify("❌ Check-out must be after check-in.", color="negative")
        return

    days = (checkout_date - checkin_date).days if checkin_date and checkout_date else 3
    selected_activities = [fid for fid in ["relaxation","adventure","family","cultural","food"] if values.get(fid)]
    selected_transports = [fid for fid in ["car","rideshare","metro","bike","walk"] if values.get(fid)]

    prompt = f"""
    Create a comprehensive travel activity guide for {dest} lasting {days} days.
    Traveler: {fname} {lname}, Party Size: {guests}, Dates: {check_in} → {check_out}
    Activities: {", ".join(selected_activities) or "Open to all experiences"}
    Transportation: {", ".join(selected_transports) or "Flexible"}
    """

    # Loading dialog
    with ui.dialog() as d, ui.card():
        ui.label("⏳ Generating guide...")
    d.open()

    # Run LLM in background thread
    def run_llm():
        llm = get_llm()
        return llm.invoke(prompt).content.strip()

    result = await anyio.to_thread.run_sync(run_llm)
    d.close()

    # Save to file
    filename = f"travel-guide-output/travel_guide_{fname}_{lname}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result)

    # Update shared state
    result_data["title"] = f"📝 Guide for {fname} {lname} in {dest} ({days} days)"
    result_data["content"] = result


    ui.navigate.to("/result")
    ui.notify(f"✅ Saved as {filename}", color="positive")

# ===== Run App =====
def main():
    widgets: WidgetRef = {}
    config = json.loads(CONFIG_JSON)
    build_ui(config, widgets)
    ui.run(reload=False, native=False)  # set native=True if pywebview installed

if __name__ in {"__main__", "__mp_main__"}:
    main()


RuntimeError: asyncio.run() cannot be called from a running event loop